In [ ]:
from scipy.optimize import curve_fit
import hyperspy.api as hys
import numpy as np
import matplotlib.pyplot as plt
import tkinter.filedialog as tkf
from tqdm import tqdm
plt.rcParams['font.family'] = 'Cambria'

import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap

color_rep = ["black", "red", "green", "blue", "orange", "purple", "yellow", "lime", 
             "cyan", "magenta", "lightgray", "peru", "springgreen", "deepskyblue", 
             "hotpink", "darkgray"]
print(len(color_rep))

rgb_rep = {"black":[1,1,1,1], "red":[1,0,0,1], "green":[0,1,0,1], "blue":[0,0,1,1], "orange":[1,0.5,0,1], "purple":[1,0,1,1],
           "yellow":[1,1,0,1], "lime":[0,1,0.5,1], "cyan":[0,1,1,1]}

custom_cmap = mcolors.ListedColormap(color_rep)
bounds = np.arange(-1, len(color_rep))
norm = mcolors.BoundaryNorm(boundaries=bounds, ncolors=len(color_rep))
sm = cm.ScalarMappable(cmap=custom_cmap, norm=norm)
sm.set_array([])

cm_rep = ["gray", "Reds", "Greens", "Blues", "Oranges", "Purples"]
print(len(cm_rep))

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [ ]:
spectrum_adr = tkf.askopenfilename()
print(spectrum_adr)

In [ ]:
spectrum = hys.load(spectrum_adr, signal_type="EELS")
print(spectrum)

si_data = spectrum.data
print(si_data.shape)

In [ ]:
step = spectrum.axes_manager[2].scale
print(step)
offset = spectrum.axes_manager[2].offset
print(offset)
e_size = spectrum.axes_manager[2].size
print(e_size)

In [ ]:
e_range = np.arange(offset, offset+e_size*step, step)
print(e_range.shape)

In [ ]:
%matplotlib widget
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.plot(e_range, np.mean(si_data, axis=(0,1)), 'k-')
fig.tight_layout()
plt.show()

In [ ]:
%matplotlib inline

In [ ]:
def gauss1(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss2(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss3(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def gauss4(x, a, c, sigma):
    return a*np.exp(-(x-c)**2/(2*sigma**2))

def two_gauss(x, a1, a2, c1, c2, sigma1, sigma2):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)

def three_gauss(x, a1, a2, a3, c1, c2, c3, sigma1, sigma2, sigma3):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)+gauss3(x, a3, c3, sigma3)

def four_gauss(x, a1, a2, a3, a4, c1, c2, c3, c4, sigma1, sigma2, sigma3, sigma4):
    return gauss1(x, a1, c1, sigma1)+gauss2(x, a2, c2, sigma2)+gauss3(x, a3, c3, sigma3)+gauss4(x, a4, c4, sigma4)

In [ ]:
fit1_bound = (0.0, 1.0, 14.0, 18.0, 0.0, 3.0)
fit2_bound = (0.0, 1.0, 21.0, 25.0, 0.0, 3.0)
fit3_bound = (0.0, 1.0, 27.0, 29.0, 0.0, 3.0)

bound_left = [fit1_bound[0], fit2_bound[0], fit3_bound[0], 
              fit1_bound[2], fit2_bound[2], fit3_bound[2],
              fit1_bound[4], fit2_bound[4], fit3_bound[4]]

bound_right = [fit1_bound[1], fit2_bound[1], fit3_bound[1], 
              fit1_bound[3], fit2_bound[3], fit3_bound[3],
              fit1_bound[5], fit2_bound[5], fit3_bound[5]]

In [ ]:
start_ev = 10.0
end_ev = 35.0
start_ind = find_nearest(e_range, start_ev)
end_ind = find_nearest(e_range, end_ev)
print(start_ev, start_ind)
print(end_ev, end_ind)

pad = 50

pbar = tqdm(total=si_data.shape[0]*si_data.shape[1])

params_map = []
for i in range(si_data.shape[0]):
    for j in range(si_data.shape[1]):
        signal = si_data[i, j]
        e_range_int = e_range[start_ind:end_ind]
        signal_int = signal[start_ind:end_ind]
        signal_int = signal_int / np.max(signal_int)

        slope = (signal_int[-1] - signal_int[0]) / (e_range_int[-1] - e_range_int[0])
        intercept = signal_int[0] - slope*e_range_int[0]
        bg_line = slope*e_range_int+intercept

        signal_int_bg_removed = signal_int - bg_line
        signal_int_bg_removed = np.append(np.zeros(pad), signal_int_bg_removed)
        signal_int_bg_removed = np.append(signal_int_bg_removed, np.zeros(pad))
        signal_int_bg_removed = signal_int_bg_removed / np.max(signal_int_bg_removed)
        e_range_int_bg_removed = np.arange(start_ev-pad*step, end_ev+pad*step, step)

        bg_line_bg_removed = slope*e_range_int_bg_removed+intercept / np.max(signal_int_bg_removed)

        popt, pcov = curve_fit(three_gauss, e_range_int_bg_removed, signal_int_bg_removed, 
                               bounds=(bound_left, bound_right))
        params_map.append(popt)
        pbar.update(1)
pbar.close()

In [ ]:
params_map = np.asarray(params_map).reshape(si_data.shape[0], si_data.shape[1], -1)
print(params_map.shape)

In [ ]:
for i in range(9):
    fig, ax = plt.subplots(1, 1, figsize=(7, 7))
    pmap = ax.imshow(params_map[:, :, i], cmap="inferno")
    ax.axis("off")
    fig.colorbar(pmap)
    fig.tight_layout()
    plt.show()